# Module 16 Challenge
## Deliverable 2

In [1]:
# Install Spark and start SparkSession

import os
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Challenge-Del2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Reviews of Kitchen Products

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [5]:
df.describe().show()

+-------+-----------+--------------------+--------------+--------------------+--------------------+------------------------+----------------+------------------+------------------+------------------+-------+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|          product_id|      product_parent|           product_title|product_category|       star_rating|     helpful_votes|       total_votes|   vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+--------------------+--------------------+------------------------+----------------+------------------+------------------+------------------+-------+-----------------+--------------------+--------------------+-----------+
|  count|    4880466|             4880466|       4880466|             4880466|             4880466|                 4880466|         4880460|           4880460|   

In [31]:
# Import additional functions
from pyspark.sql.functions import to_date, col, count, when

In [7]:
# Create Vine Table
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3DT59XH7HXR9K|          5|            0|          0|   N|                Y|
|R1LFS11BNASSU8|          5|            0|          1|   N|                Y|
|R296RT05AG0AF6|          5|            0|          0|   N|                Y|
|R3V37XDZ7ZCI3L|          5|            0|          1|   N|                Y|
|R14GU232NQFYX2|          5|            0|          0|   N|                Y|
| RZQH4V7L2O1PL|          1|            1|          1|   N|                Y|
|R1F8JMOSPJ3KO7|          5|            0|          0|   N|                Y|
|R1ZISGY2BWW4Z5|          5|            0|          0|   N|                Y|
|R17PW4I3AE5WZW|          5|            0|          0|   N|                Y|
|R3D93G1KTP6A8P|          3|            0|          0|   N|     

In [8]:
# Filter Vine Table for more than 20 votes
filter1_df = vine_df.filter("total_votes>20")
filter1_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FB6BERWPEIJP|          4|           40|         43|   N|                Y|
|R1D4Z38STRDQXK|          5|           53|         56|   N|                Y|
|R1XMWJZICINIFX|          3|           20|         21|   N|                Y|
|R20QKY1GABXFLM|          1|          272|        297|   N|                Y|
|R3GZ7CK2O1PPG0|          2|           10|         30|   N|                Y|
|R3DH22AA5WGLLS|          5|           30|         30|   N|                N|
|R1E0E5EFZSLJCS|          1|           66|         80|   N|                Y|
|R1TXZQWEHYWEWN|          2|           48|         51|   N|                Y|
| RQQAI8YL3UCY2|          5|           23|         25|   N|                Y|
|R3PZNAQREFUNJ7|          4|           28|         35|   N|     

In [9]:
filter1_df.describe().show()

+-------+--------------+------------------+-----------------+------------------+------+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|       total_votes|  vine|verified_purchase|
+-------+--------------+------------------+-----------------+------------------+------+-----------------+
|  count|        101229|            101229|           101229|            101229|101229|           101229|
|   mean|          null|3.4135079868417155|61.10050479605647| 68.53277222930188|  null|             null|
| stddev|          null|1.6828907400436925|146.7081772848936|152.31024230683553|  null|             null|
|    min|R1005RBHBUJ9PQ|                 1|                0|                21|     N|                N|
|    max| RZZZSRMJ6EVKF|                 5|            11173|             11501|     Y|                Y|
+-------+--------------+------------------+-----------------+------------------+------+-----------------+



In [11]:
# Filter Table for helpful votes >50%
filter2_df = filter1_df.withColumn("percent_helpful", col("helpful_votes")/col("total_votes")).filter(col("percent_helpful")>=0.5)
filter2_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3FB6BERWPEIJP|          4|           40|         43|   N|                Y|0.9302325581395349|
|R1D4Z38STRDQXK|          5|           53|         56|   N|                Y|0.9464285714285714|
|R1XMWJZICINIFX|          3|           20|         21|   N|                Y|0.9523809523809523|
|R20QKY1GABXFLM|          1|          272|        297|   N|                Y|0.9158249158249159|
|R3DH22AA5WGLLS|          5|           30|         30|   N|                N|               1.0|
|R1E0E5EFZSLJCS|          1|           66|         80|   N|                Y|             0.825|
|R1TXZQWEHYWEWN|          2|           48|         51|   N|                Y|0.9411764705882353|
| RQQAI8YL3UCY2|          5|  

In [12]:
filter2_df.describe().show()

+-------+--------------+------------------+-----------------+-----------------+-----+-----------------+-------------------+
|summary|     review_id|       star_rating|    helpful_votes|      total_votes| vine|verified_purchase|    percent_helpful|
+-------+--------------+------------------+-----------------+-----------------+-----+-----------------+-------------------+
|  count|         93468|             93468|            93468|            93468|93468|            93468|              93468|
|   mean|          null|3.5486583643599947|65.51732143621346|71.04407925707194| null|             null| 0.9035509391476819|
| stddev|          null|1.6417372000871353|151.8269277718066|158.0790786751467| null|             null|0.09302274302073417|
|    min|R1005RBHBUJ9PQ|                 1|               11|               21|    N|                N|                0.5|
|    max| RZZZSRMJ6EVKF|                 5|            11173|            11501|    Y|                Y|                1.0|
+-------

In [15]:
# Filter for Vine program reviews
Vine_df = filter2_df.filter(col("vine")=="Y")
Vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1Z71RW4J9IK93|          5|           20|         22|   Y|                N|0.9090909090909091|
|R3FVB5QI11KI9Q|          4|          192|        200|   Y|                N|              0.96|
|R2G027YBMVXV6Y|          5|           39|         48|   Y|                N|            0.8125|
|R1QGBAN7BMGWRR|          5|          121|        129|   Y|                N| 0.937984496124031|
|R2YVVJ9NOPNX50|          4|           36|         41|   Y|                N|0.8780487804878049|
|R1XH1LK1FWX3OS|          4|          214|        238|   Y|                N|0.8991596638655462|
|R38LSQ71G2IZGS|          5|           26|         29|   Y|                N| 0.896551724137931|
|R2FLITQVKWXDF4|          3|  

In [16]:
# Filter for Non-Vine reviews
NoVine_df = filter2_df.filter(col("vine")=="N")
NoVine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3FB6BERWPEIJP|          4|           40|         43|   N|                Y|0.9302325581395349|
|R1D4Z38STRDQXK|          5|           53|         56|   N|                Y|0.9464285714285714|
|R1XMWJZICINIFX|          3|           20|         21|   N|                Y|0.9523809523809523|
|R20QKY1GABXFLM|          1|          272|        297|   N|                Y|0.9158249158249159|
|R3DH22AA5WGLLS|          5|           30|         30|   N|                N|               1.0|
|R1E0E5EFZSLJCS|          1|           66|         80|   N|                Y|             0.825|
|R1TXZQWEHYWEWN|          2|           48|         51|   N|                Y|0.9411764705882353|
| RQQAI8YL3UCY2|          5|  

## Analyze Vine and Non-Vine Reviews
- Total number
- Number of 5-star reviews
- Percentage of 5-star reviews

In [54]:
# Vine Reviews
Vine_df.agg(count("review_id"),count(when(col("star_rating") == "5", 1))).\
withColumnRenamed("count(review_id)","Total Reviews").\
withColumnRenamed("count(CASE WHEN (star_rating = 5) THEN 1 END)","Number of 5 Star Ratings").\
withColumn("Percent 5 Star Reviews", col("Number of 5 Star Ratings")/col("Total Reviews")*100).show()

+-------------+------------------------+----------------------+
|Total Reviews|Number of 5 Star Ratings|Percent 5 Star Reviews|
+-------------+------------------------+----------------------+
|         1160|                     489|      42.1551724137931|
+-------------+------------------------+----------------------+



In [60]:
# Confirming count of all star ratings
Vine_df.groupby("star_rating").count().show()


+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1|   42|
|          3|  214|
|          5|  489|
|          4|  347|
|          2|   68|
+-----------+-----+



In [55]:
# Non-Vine Reviews
NoVine_df.agg(count("review_id"),count(when(col("star_rating") == "5", 1))).\
withColumnRenamed("count(review_id)","Total Reviews").\
withColumnRenamed("count(CASE WHEN (star_rating = 5) THEN 1 END)","Number of 5 Star Ratings").\
withColumn("Percent 5 Star Reviews", col("Number of 5 Star Ratings")/col("Total Reviews")*100).show()

+-------------+------------------------+----------------------+
|Total Reviews|Number of 5 Star Ratings|Percent 5 Star Reviews|
+-------------+------------------------+----------------------+
|        92308|                   43247|     46.85076049746501|
+-------------+------------------------+----------------------+



In [56]:
# Confirming count of all star ratings - non-vine reviews
NoVine_df.groupby("star_rating").count().show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1|21201|
|          3| 8133|
|          5|43247|
|          4|12872|
|          2| 6855|
+-----------+-----+

